In [ ]:
import numpy as np
import tensorflow as tf
import keras
import torch
import os, glob
import pickle as pkl
from keras import layers
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Input, Bidirectional, LSTM, Activation, TimeDistributed, BatchNormalization
from sklearn.model_selection import train_test_split
from keras.utils.vis_utils import plot_model
import tensorflow.keras.backend as K
import pandas as pd
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.9 MB/s eta 0:00:00


In [ ]:
import os
import torch

In [ ]:
from transformers import BertTokenizer, BertModel #RobertaTokenizer, RobertaModel, XLMRobertaTokenizer, TFXLMRobertaModel
import numpy as np
import glob, os
from os import path
import pickle as pkl
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states = True,)

In [ ]:
path = '/content/drive/MyDrive/funny_text'
save_path = '/content/drive/MyDrive/Colab Notebooks'

In [ ]:
count=0
text=[]
X_text=[]


In [ ]:
filenames = glob.glob(os.path.join(path, '*.txt'))
filenames = sorted(filenames)

In [ ]:
for i in filenames:
  print(i)

/content/drive/MyDrive/funny_text/AJ_TP_text_01.txt
/content/drive/MyDrive/funny_text/AJ_TP_text_02.txt
/content/drive/MyDrive/funny_text/AJ_TP_text_03.txt
/content/drive/MyDrive/funny_text/AJ_TP_text_04.txt
/content/drive/MyDrive/funny_text/AJ_TP_text_05.txt
/content/drive/MyDrive/funny_text/AJ_TP_text_06.txt
/content/drive/MyDrive/funny_text/AJ_TP_text_07.txt
/content/drive/MyDrive/funny_text/AJ_TP_text_08.txt
/content/drive/MyDrive/funny_text/AJ_TP_text_09.txt
/content/drive/MyDrive/funny_text/AJ_TP_text_10.txt
/content/drive/MyDrive/funny_text/AJ_TP_text_11.txt
/content/drive/MyDrive/funny_text/AJ_TP_text_12.txt
/content/drive/MyDrive/funny_text/AJ_TP_text_13.txt
/content/drive/MyDrive/funny_text/AJ_TP_text_14.txt
/content/drive/MyDrive/funny_text/AJ_TP_text_15.txt
/content/drive/MyDrive/funny_text/AJ_TP_text_16.txt
/content/drive/MyDrive/funny_text/AJ_TP_text_17.txt
/content/drive/MyDrive/funny_text/AJ_TP_text_18.txt
/content/drive/MyDrive/funny_text/AJ_TP_text_19.txt
/content/dri

In [ ]:
for i in filenames:
  print(i)
  with open(i,'r') as f:
    text1=f.read()
    text_list=text1.strip().split(' ')
    final_text_list = ["[CLS]"]

    for word in text_list:
      final_text_list.append(word)

    final_text=" ".join(final_text_list)

    f.close()

    tokenized_text = tokenizer.tokenize(final_text)

    if len(tokenized_text) > 511:
      tokenized_text = tokenized_text[:511]
    tokenized_text.append('[SEP]')

    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    model = model.eval()

    with torch.no_grad():
            outputs = model(tokens_tensor, segments_tensors)
            hidden_states = outputs[2]
    token_embeddings = torch.stack(hidden_states, dim=0)
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
    token_embeddings = token_embeddings.permute(1, 0, 2)
    token_vecs_sum=[]

    for token in token_embeddings:
      sum_vec = torch.sum(token[-4:], dim=0)
      sum_vec_np = sum_vec.numpy()
      token_vecs_sum.append(sum_vec_np)

    if len(token_vecs_sum) < 512:
       for j in range(512-len(token_vecs_sum)):
        padding_vec = np.zeros(768)
        token_vecs_sum.append(padding_vec)

    token_vecs_sum = torch.stack([torch.sum(token[-4:], dim=0) for token in token_embeddings])
    pad_token_vecs_sum = torch.nn.utils.rnn.pad_sequence(token_vecs_sum, batch_first=True, padding_value=0)

    bert_file = i.split('/')[-1]
    bert_file = bert_file.split('.')[0]
    bert_file = save_path + bert_file + '_bb.pkl'
    with open(bert_file, 'wb') as f:
      pkl.dump(pad_token_vecs_sum, f)

    X_text.append(pad_token_vecs_sum)
  count = count + 1

print(count)



/content/drive/MyDrive/funny_text/AJ_TP_text_01.txt
/content/drive/MyDrive/funny_text/AJ_TP_text_02.txt
/content/drive/MyDrive/funny_text/AJ_TP_text_03.txt
/content/drive/MyDrive/funny_text/AJ_TP_text_04.txt
/content/drive/MyDrive/funny_text/AJ_TP_text_05.txt
/content/drive/MyDrive/funny_text/AJ_TP_text_06.txt
/content/drive/MyDrive/funny_text/AJ_TP_text_07.txt
/content/drive/MyDrive/funny_text/AJ_TP_text_08.txt
/content/drive/MyDrive/funny_text/AJ_TP_text_09.txt
/content/drive/MyDrive/funny_text/AJ_TP_text_10.txt
/content/drive/MyDrive/funny_text/AJ_TP_text_11.txt
/content/drive/MyDrive/funny_text/AJ_TP_text_12.txt
/content/drive/MyDrive/funny_text/AJ_TP_text_13.txt
/content/drive/MyDrive/funny_text/AJ_TP_text_14.txt
/content/drive/MyDrive/funny_text/AJ_TP_text_15.txt
/content/drive/MyDrive/funny_text/AJ_TP_text_16.txt
/content/drive/MyDrive/funny_text/AJ_TP_text_17.txt
/content/drive/MyDrive/funny_text/AJ_TP_text_18.txt
/content/drive/MyDrive/funny_text/AJ_TP_text_19.txt
/content/dri

In [ ]:
path2 = '/content/drive/MyDrive/Colab Notebooks'
with open(path2 + 'X_txt_bert.pkl', 'wb') as f:
  pkl.dump(X_text, f)

In [ ]:
with open(path2 + 'X_txt_bert.pkl', 'rb') as f:
    X_text_bert = pkl.load(f)
input = keras.Input(shape=(512, 768,), name = "Text Embeddings")
x = layers.Bidirectional(layers.LSTM(256, return_sequences=True), name = "Text_Layer")(input)
a = layers.Bidirectional(layers.LSTM(128), name = "Text_Layer2")(x)
c = layers.Dense(100, activation="sigmoid", name = "Text")(a)
output = layers.Dense(5, activation="softmax", name = "Output")(c)
model2 = keras.Model(input, output)
model2.compile("adam","categorical_crossentropy" , metrics=["accuracy"])

xtrain, xtest= train_test_split(X_text_bert, train_size=0.9, random_state=10)

X_train_bert = K.constant(xtrain)
X_test_bert = K.constant(xtest_bert)


ValueError: ignored